## GUIA 2

## Ejercicio 1


![image.png](enunciados/ej1.png)

Comenzamos definiendo las variables y los datos que nos da el enunciado

Y : "Bit emitido"

X : "Bit recibido"

P(Y = 1) = 3/4 : Probabilidad de que el bit emitido sea 1

P(Y = 0) = 1/4 : Probabilidad de que el bit emitido sea 0

X|Y=0 ~ N(0,4) : Distribucion del bit recibido, dado que se emitio un 0

X|Y=1 ~ N(0,1) : Distribucion del bit recibido, dado que se emitio un 0

a) 

$p_X(x) = P(Y = 0) \cdot p_{X|Y=0}(x) + P(Y = 1) \cdot p_{X|Y=1}(x)$


$p_{X|Y=0}(x) = \frac{1}{ 2 \sqrt \pi} \cdot exp(-\frac{x^2}{8})$

$p_{X|Y=1}(x) = \frac{1}{ \sqrt{2 \pi}} \cdot exp(-\frac{x^2}{2})$


